In [14]:
import pandas as pd
import seaborn as sns
import pytz

In [28]:
new_df2 = pd.read_csv('data/dsa_data.csv')
new_df2['Datetime'] = pd.to_datetime(new_df2['Datetime'])
new_df2.dropna(subset = ['User Number'], inplace=True)
new_df2 = new_df2[(new_df2['Direction']=='Entry') | (new_df2['Direction'] == 'Exit')]
sgt_timezone = pytz.timezone('Asia/Singapore')
new_df2['Datetime'] = new_df2['Datetime'].dt.tz_convert(sgt_timezone)
new_df2['Entry'] = new_df2['Direction'].replace({'Entry':1, 'Exit':-1})
new_df2 = new_df2.sort_values('Datetime')
new_df2['Date'] = new_df2['Datetime'].dt.date
new_df2

,Datetime,User Number,Broad Category,Library,Method,Direction,Entry,Date
0,2023-01-25 07:29:13+08:00,2483.0,Library Professional,Central Library,Card,Entry,1,2023-01-25
1,2023-01-25 07:32:15+08:00,4372.0,Library Professional,Central Library,Card,Entry,1,2023-01-25
2,2023-01-25 07:32:35+08:00,2.0,Others,Central Library,Card,Entry,1,2023-01-25
3,2023-01-25 07:32:45+08:00,7405.0,Others,Central Library,Card,Entry,1,2023-01-25
4,2023-01-25 07:32:49+08:00,2182.0,Others,Central Library,Card,Entry,1,2023-01-25
...,...,...,...,...,...,...,...,...
127874,2023-04-29 17:00:52+08:00,7423.0,Library Professional,Central Library,Card,Exit,-1,2023-04-29
127913,2023-04-29 17:36:09+08:00,6427.0,Undergraduate,Central Library,Card,Entry,1,2023-04-29
127914,2023-04-29 17:37:28+08:00,4318.0,Administrative Staff,Central Library,Card,Entry,1,2023-04-29
127917,2023-04-29 17:38:18+08:00,4318.0,Administrative Staff,Central Library,Card,Exit,-1,2023-04-29


In [16]:
unique_broad_categories = new_df2['Broad Category'].unique()
unique_broad_categories

array(['Library Professional', 'Others', 'Teaching Staff', 'Postgraduate',
       'Undergraduate', 'Special Student', 'Administrative Staff', nan,
       'Research Staff', 'External Members/Alumni', 'Complimentary',
       'SCALE Students', 'Non-Academic Staff', 'External Members',
       'Corporate Card'], dtype=object)

In [17]:
new_df2 = new_df2[new_df2['Broad Category']!= 'Library Professional']
new_df2

,Datetime,User Number,Broad Category,Library,Method,Direction,Entry
2,2023-01-25 07:32:35+08:00,2.0,Others,Central Library,Card,Entry,1
3,2023-01-25 07:32:45+08:00,7405.0,Others,Central Library,Card,Entry,1
4,2023-01-25 07:32:49+08:00,2182.0,Others,Central Library,Card,Entry,1
5,2023-01-25 07:32:53+08:00,8453.0,Others,Central Library,Card,Entry,1
6,2023-01-25 07:40:10+08:00,30.0,Others,Central Library,Card,Entry,1
...,...,...,...,...,...,...,...
127862,2023-04-29 16:54:20+08:00,5348.0,Postgraduate,Central Library,QR,Exit,-1
127913,2023-04-29 17:36:09+08:00,6427.0,Undergraduate,Central Library,Card,Entry,1
127914,2023-04-29 17:37:28+08:00,4318.0,Administrative Staff,Central Library,Card,Entry,1
127917,2023-04-29 17:38:18+08:00,4318.0,Administrative Staff,Central Library,Card,Exit,-1


In [31]:
#statistic on the duration of stay
entry_exit_list = []
for date, group in new_df2.groupby(['Date', 'User Number']):
    user_number = date[1]
    entry_times = group[group['Direction'] == 'Entry']['Datetime'].tolist()
    exit_times = group[group['Direction'] == 'Exit']['Datetime'].tolist()
    entry_exit_list.append({'User Number': user_number, 'Date': date[0], 'Entry Times': entry_times, 'Exit Times': exit_times})
entry_exit_list


[{'User Number': 1.0,
  'Date': datetime.date(2023, 1, 25),
  'Entry Times': [Timestamp('2023-01-25 07:48:10+0800', tz='Asia/Singapore'),
   Timestamp('2023-01-25 11:13:24+0800', tz='Asia/Singapore'),
   Timestamp('2023-01-25 14:17:25+0800', tz='Asia/Singapore'),
   Timestamp('2023-01-25 14:50:26+0800', tz='Asia/Singapore')],
  'Exit Times': [Timestamp('2023-01-25 14:37:26+0800', tz='Asia/Singapore')]},
 {'User Number': 2.0,
  'Date': datetime.date(2023, 1, 25),
  'Entry Times': [Timestamp('2023-01-25 07:32:35+0800', tz='Asia/Singapore'),
   Timestamp('2023-01-25 08:19:02+0800', tz='Asia/Singapore'),
   Timestamp('2023-01-25 10:36:39+0800', tz='Asia/Singapore'),
   Timestamp('2023-01-25 13:03:46+0800', tz='Asia/Singapore'),
   Timestamp('2023-01-25 13:26:32+0800', tz='Asia/Singapore'),
   Timestamp('2023-01-25 16:50:12+0800', tz='Asia/Singapore')],
  'Exit Times': [Timestamp('2023-01-25 08:16:38+0800', tz='Asia/Singapore'),
   Timestamp('2023-01-25 09:44:57+0800', tz='Asia/Singapore'),